In [7]:
import numpy as np 
import pandas as pd 
import polars as pl
import os
import optuna
import re
import warnings
from sklearn.model_selection import KFold, train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import lightgbm as lgb
from lightgbm import LGBMRegressor
from lightgbm import log_evaluation, early_stopping

warnings.simplefilter('ignore')

In [8]:
train = pd.read_csv('/home/baebro/kaggle_ws/car price/data/train.csv')
test = pd.read_csv('/home/baebro/kaggle_ws/car price/data/test.csv')
orig = pd.read_csv('/home/baebro/kaggle_ws/car price/data/used_cars.csv')


orig[['milage', 'price']] = orig[['milage', 'price']].apply(
    lambda col: col.map(lambda x: int(''.join(re.findall(r'\d+', str(x))))))

train = pd.concat([train, orig], ignore_index=True)

In [9]:
def fill_nan_values(df):
    
    freq = 100
    
    categoricl_colums = ['brand','model','fuel_type','engine','transmission','ext_col','int_col','accident','clean_title']
    re_ = ['model','engine','transmission','ext_col','int_col']
    
    for column in re_:
        df.loc[df[column].value_counts(dropna=False)[df[column]].values < freq, column] = "noise"
        
    for column in categoricl_colums:
        df[column] = df[column].fillna('missing')
        df[column] = df[column].astype('category')
        
    return df

train  = fill_nan_values(train)
test   = fill_nan_values(test)

In [10]:
X_train = train.drop(columns=["id", "price"])
y_train = train["price"]
X_test = test.drop(columns=["id"])

In [11]:
# def cross_validate_model_l(model, X_train, y_train, X_test, params, n_splits=5):

#     callbacks = [log_evaluation(period=150), early_stopping(stopping_rounds=200)]
#     cv = KFold(n_splits=n_splits, shuffle=True, random_state=42)
#     val_scores = []
#     test_preds = np.zeros((len(X_test), n_splits), dtype=np.float32)

#     for fold, (train_ind, valid_ind) in enumerate(cv.split(X_train)):
#         # Data splitting
#         X_fold_train = X_train.iloc[train_ind]
#         y_fold_train = y_train.iloc[train_ind]
#         X_val = X_train.iloc[valid_ind]
#         y_val = y_train.iloc[valid_ind]
        
#         clf = model(**params)
#         clf.fit(X_fold_train, y_fold_train, eval_set=[(X_val, y_val)], callbacks=callbacks)
        
#         test_preds[:, fold] = clf.predict(X_test)

#         print("-" * 50)
#         print(f"Fold {fold + 1} predictions: {test_preds[:, fold]}")

#     test_preds = np.mean(test_preds, axis=1)
#     return clf, test_preds

In [ ]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor
from sklearn.metrics import make_scorer, mean_squared_error
from sklearn.model_selection import KFold

# 하이퍼파라미터 탐색 범위 설정
param_grid = {
    'num_leaves': [20, 50, 100, 200],
    'max_depth': [5, 10, 15, 20],
    'learning_rate': [0.01, 0.05, 0.1],
    'n_estimators': [1000, 5000, 10000],
    'subsample': [0.5, 0.7, 0.9],
    'colsample_bytree': [0.5, 0.7, 0.9],
    'reg_alpha': [0.0, 0.1, 1.0],
    'reg_lambda': [0.0, 0.1, 1.0],
    'min_data_in_leaf': [20, 50, 100]
}

# LightGBM 모델 초기화
lgb_model = LGBMRegressor(random_state=42)

# RMSE를 음수로 반환하는 사용자 정의 스코어 함수
rmse_scorer = make_scorer(mean_squared_error, greater_is_better=False, squared=False)

# K-Fold 교차 검증 설정 (5개의 폴드로 나눔)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# GridSearchCV 초기화
grid_search = GridSearchCV(
    lgb_model,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=kf,  # K-Fold 교차 검증 설정
    verbose=2,
    n_jobs=-1  # 병렬 처리
)

# 훈련 데이터에 대해 GridSearchCV 수행
grid_search.fit(X_train, y_train)

# 최적의 하이퍼파라미터 출력
print("Best Hyperparameters found by GridSearchCV:")
print(grid_search.best_params_)

# 최적의 모델로 예측
y_test_pred = grid_search.predict(X_test)

Fitting 5 folds for each of 34992 candidates, totalling 174960 fits


In [ ]:
# 튜닝된 하이퍼파라미터 가져오기
best_params = grid_search.best_params_  # RandomizedSearchCV를 통해 얻은 최적 하이퍼파라미터
# 또는
# best_params = grid_search.best_params_  # GridSearchCV를 통해 얻은 최적 하이퍼파라미터

# 최적의 하이퍼파라미터로 LightGBM 모델 생성
final_model = LGBMRegressor(**best_params)

# 전체 훈련 데이터를 사용하여 모델 학습
final_model.fit(X_train, y_train)

# 테스트 데이터에 대해 예측 수행
y_test_pred = final_model.predict(X_test)

# 예측 결과 확인 (처음 10개의 예측값 출력)
print("Test Predictions: ", y_test_pred[:10])

# 성능 평가 (RMSE)
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
print(f"Test RMSE: {rmse:.4f}")

In [33]:
lgb_result =  pd.read_csv('/home/baebro/kaggle_ws/kaggle_competitions/Regression of Used Car Prices/data/sample_submission.csv')
lgb_result['price'] = y_test_pred.astype(np.float32)
#xgb_result.to_csv('result_xgb.csv', index=False)
lgb_result
lgb_result.to_csv('submission_lgb_w_original_Optuna_tunning4.csv', index=False)